In [46]:
import networkx as nx
from collections import deque
import matplotlib.pyplot as plt

############################################
# 同一の val 値を持つノードを返す関数を定義
def get_nodes_with_val(G, val):
    return [node for node, data in G.nodes(data=True) if data.get('val') == val]

############################################
# ノードの val 値を変更する関数を定義
def set_node_val(G, node, new_val):
    if node in G.nodes():
        G.nodes[node]['val'] = new_val
#        print(f"Node {node} val updated to {new_val}")
    else:
        print(f"Node {node} not found in graph G")

############################################
def mVG(G, vj, nodes):
    xi = set()

    # Check if vj exists in G
    if (0, vj) not in G.nodes():
        return xi

    distances = {node: float('inf') for node in G.nodes()}
    distances[(0, vj)] = 0
    queue = deque([(0, vj)])

    while queue:
        current = queue.popleft()
        current_dist = distances[current]

        for neighbor in G[current]:
            if distances[neighbor] > current_dist + 1:
                distances[neighbor] = current_dist + 1
                queue.append(neighbor)

    found = False
    for d in range(1, len(G.nodes())):
        for node in G.nodes():
            if distances[node] == d:
                all_neighbors_found = all(distances[(0, n)] != float('inf') for n in nodes)
                if all_neighbors_found:
                    v_star = node
                    found = True
                    break
        if found:
            break

    if not found:
        return xi

    xi.add(v_star)

    for v in nodes:
        if v != vj:
            distances = {node: float('inf') for node in G.nodes()}
            queue = deque(xi)
            distances[v_star] = 0

            while queue:
                current = queue.popleft()
                current_dist = distances[current]

                for neighbor in G[current]:
                    if distances[neighbor] > current_dist + 1:
                        distances[neighbor] = current_dist + 1
                        queue.append(neighbor)

            shortest_path_node = min((node for node in distances if distances[node] != float('inf')), key=distances.get)
            if distances[shortest_path_node] != float('inf'):
                xi.add(shortest_path_node)

    return xi

# テスト用のグラフ H と G の初期化（前述のコードを使用）

############################################
# グラフ H を初期化
H = nx.Graph()
H.add_nodes_from([0, 1, 2, 3, 4])
H.add_edges_from([(0, 1), (0, 2), (0, 4), (1, 2), (2, 3), (2, 4), (3, 4)])
n = H.number_of_nodes()   # グラフHのノード数
H_nodes = list(H.nodes()) # グラフHのノードリスト

############################################
# キンググラフ G に val 属性を追加し、グラフ H のノードの値を格納
def InitializeG(n, H_nodes):
    G = nx.grid_2d_graph(n-1, n)

    for x, y in G.nodes():
        if x == 0:  # 第0行の場合
            G.nodes[(x, y)]['val'] = H_nodes[y]
        else:
            # x + y が偶数の場合
            if (x + y) % 2 == 0:
                G.nodes[(x, y)]['val'] = G.nodes[(x - 1, max(0, y - 1))]['val']
            # x + y が奇数の場合
            else:
                G.nodes[(x, y)]['val'] = G.nodes[(x - 1, min(n - 1, y + 1))]['val']
    return G

G = InitializeG(n, H_nodes)

############################################
# 格子状レイアウトでグラフを描画
def Show_KingGraph(K):
    # yを横、-xを縦にすることで正しい表現（行, 列）になる
    pos = {(x, y): (y, -x) for x, y in K.nodes()}

    # ノードラベルを設定
    # labels = {(x, y): f"({x},{y})" for x, y in G.nodes()}
    labels = {(x, y): f"({x},{y})\n{G.nodes[(x, y)]['val']}" for x, y in G.nodes()}

    plt.figure(figsize=(8, 6))
    nx.draw(K, pos, labels=labels, with_labels=True, node_size=1200, node_color='skyblue', font_size=12, font_color='black')
    plt.title('King graph')
    plt.show()

# Show_KingGraph(G)

############################################
vi = 0  # テストテストテスト
print(f"vi = {vi}: ")

# 6行目
for node in get_nodes_with_val(G, H_nodes[vi]):
    set_node_val(G, node, '')

for vj in H.neighbors(vi):
    print(f"vj = {vj}: ", end='')
    print(set(H.neighbors(vj)) - {vi})
    if (0, vj) in G.nodes():
        xi = mVG(G, vj, set(H.neighbors(vj)) - {vi})
        print(f"xi = {xi}")
    else:
        print(f"Node (0, {vj}) not in G")


vi = 0: 
vj = 1: {2}
xi = {(0, 0)}
vj = 2: {1, 3, 4}
xi = {(0, 1)}
vj = 4: {2, 3}
xi = {(0, 3)}
